In [1]:
import pandas as pd

dataset_dir = 'datasets'

# read the dataset
df = pd.read_csv(f'{dataset_dir}/podcast_with_summary_train.csv')
df.head()

,text,text_short,summary,summary2
0,The following is a conversation with Yosha Bac...,The following is a conversation with Yosha Bac...,Lex Friedman interviews Yosha Bach on intellig...,"Yosha Bach discusses intelligence, cognition, ..."
1,The following is a conversation with John Hopf...,The following is a conversation with John Hopf...,"John Hopfield, renowned physicist and biologis...","John Hopfield, a Princeton professor, applied ..."
2,The following is a conversation with Ilya Sots...,The following is a conversation with Ilya Sots...,A conversation with Ilya Sotskever on deep lea...,A conversation with Ilya Sotskever on deep lea...
3,The following is a conversation with Travis Ol...,The following is a conversation with Travis Ol...,Conversation with Travis Oliphant on his impac...,"Travis Oliphant's work with NumPy, SciPy, and ..."
4,"Well, the source of energy at the origin of li...","Well, the source of energy at the origin of li...",Nick Lane discusses life's energy origins from...,Nick Lane discusses the origin of life and ene...


In [2]:
# find the summary with the longest length
df['summary_length'] = df['summary'].apply(lambda x: len(x))
print(df['summary_length'].max())

# find the text with the longest length
df['text_length'] = df['text_short'].apply(lambda x: len(x))
print(df['text_length'].max())

175
1024


In [3]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

dataset = Dataset.from_pandas(df)

c:\Users\paula\.conda\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "t5-small"
# 60.5M params
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["text_short"], max_length=1024, truncation=True, padding="max_length"
    )
    # Tokenize targets
    labels = tokenizer(
        examples["summary"], max_length=200, truncation=True, padding="max_length"
    )
    
    # Replace padding token id's of the labels by -100
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply the preprocessing
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map: 100%|██████████| 255/255 [00:00<00:00, 2773.96 examples/s]


In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./t5_output",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=100,  
    logging_dir='./logs', 
    report_to="none", 
)

c:\Users\paula\.conda\envs\llama\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
from transformers import DataCollatorForSeq2Seq, Trainer
import numpy as np

data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=-100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\paula\AppData\Local\Temp\ipykernel_27928\4262019623.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()

  2%|▏         | 101/6400 [00:08<08:09, 12.86it/s]

{'loss': 3.0088, 'grad_norm': 9.208634376525879, 'learning_rate': 1.96875e-05, 'epoch': 1.56}


  3%|▎         | 201/6400 [00:15<07:58, 12.96it/s]

{'loss': 2.4699, 'grad_norm': 5.115659713745117, 'learning_rate': 1.9375e-05, 'epoch': 3.12}


  5%|▍         | 301/6400 [00:23<07:53, 12.87it/s]

...

100%|██████████| 6400/6400 [08:39<00:00, 12.88it/s]

{'loss': 1.0596, 'grad_norm': 7.01862096786499, 'learning_rate': 0.0, 'epoch': 100.0}


100%|██████████| 6400/6400 [08:40<00:00, 12.30it/s]

{'train_runtime': 520.1561, 'train_samples_per_second': 49.024, 'train_steps_per_second': 12.304, 'train_loss': 1.4070651566982268, 'epoch': 100.0}


TrainOutput(global_step=6400, training_loss=1.4070651566982268, metrics={'train_runtime': 520.1561, 'train_samples_per_second': 49.024, 'train_steps_per_second': 12.304, 'total_flos': 6902431875072000.0, 'train_loss': 1.4070651566982268, 'epoch': 100.0})

In [8]:
def run_inference(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=1024
    )
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to GPU

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=200, 
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [9]:
test_df = pd.read_csv(f'{dataset_dir}/podcast_with_summary_test.csv')

test_text = test_df.text_short[0]
print("Original:", test_text)

summary = run_inference(test_text)

print("Predicted:", summary)
print("Actual:", test_df.summary[0])

Original: The following is a conversation with Andrew Ng, one of the most impactful educators, researchers, innovators, and leaders in artificial intelligence and technology space in general. He cofounded Coursera and Google Brain, launched Deep Learning AI, Landing AI, and the AI Fund, and was the chief scientist at Baidu. As a Stanford professor and with Coursera and Deep Learning AI, he has helped educate and inspire millions of students, including me. This is the Artificial Intelligence Podcast. If you enjoy it, subscribe on YouTube, give it five stars on Apple Podcast, support it on Patreon, or simply connect with me on Twitter at Lex Friedman, spelled F R I D M A N. As usual, I'll do one or two minutes of ads now and never any ads in the middle that can break the flow of the conversation. I hope that works for you and doesn't hurt the listening experience. This show is presented by Cash App, the number one finance app in the App Store. When you get it, use code LEXPODCAST.
Predic

In [10]:
import time
from SharedUtils import evaluate_and_save_metrics

# iterate over all rows in test_df
def evaluate_df(df, name):
    reference_summaries = []
    predicted_summaries = []
    total_time = 0
    for i, row in df.iterrows():
        test_text = row.text_short
        
        start_time = time.time()
        summary = run_inference(test_text)
        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        
        reference_summaries.append(row.summary)
        predicted_summaries.append(summary)
    
    model_name = "t5-small"

    rouge_results, results_bleu = evaluate_and_save_metrics(
        model_name,
        name,
        "finetuned",
        reference_summaries,
        predicted_summaries,
        total_time
    )

    results_df = pd.DataFrame({
        'summary': reference_summaries,
        'summary_tuned': predicted_summaries
    })
    results_df.to_csv(f"./results/{model_name}/{name}/summaries.csv")

    print(rouge_results)
    print(results_bleu)
    print("Total time (seconds):", total_time)
    print("Total time (minutes): ", total_time / 60)
    

In [11]:
evaluate_df(test_df, "test_dataset")

{'rouge1': 0.42616871051572736, 'rouge2': 0.186207504189896, 'rougeL': 0.346356334240144, 'rougeLsum': 0.3457898933348439}
{'bleu': 0.11286839272559764, 'precisions': [0.4532095901005414, 0.1757526444263629, 0.07296137339055794, 0.0326975476839237], 'brevity_penalty': 0.961324598499326, 'length_ratio': 0.9620535714285714, 'translation_length': 1293, 'reference_length': 1344}
Total time (seconds): 13.28156065940857
Total time (minutes):  0.22135934432347615


In [12]:
train_df = pd.read_csv(f'{dataset_dir}/podcast_with_summary_train.csv')
evaluate_df(train_df, 'train_dataset')

{'rouge1': 0.5203022353242932, 'rouge2': 0.309463613303818, 'rougeL': 0.47092757451794753, 'rougeLsum': 0.47013257034315753}
{'bleu': 0.2432040962010115, 'precisions': [0.556078431372549, 0.30691434468524253, 0.1934640522875817, 0.1344867358708189], 'brevity_penalty': 0.9421339254342243, 'length_ratio': 0.9437453737971873, 'translation_length': 5100, 'reference_length': 5404}
Total time (seconds): 56.27474498748779
Total time (minutes):  0.9379124164581298


In [13]:
whole_df = pd.read_csv(f'{dataset_dir}/podcast_with_summary.csv')
evaluate_df(whole_df, 'whole_dataset')

{'rouge1': 0.4924141255332442, 'rouge2': 0.27201495504419093, 'rougeL': 0.43571749420356526, 'rougeLsum': 0.43675843020484584}
{'bleu': 0.21425726757200045, 'precisions': [0.5198959289868381, 0.267739340305712, 0.16095658073270014, 0.10722610722610723], 'brevity_penalty': 0.9677787711511946, 'length_ratio': 0.9682868998221695, 'translation_length': 6534, 'reference_length': 6748}
Total time (seconds): 68.71482419967651
Total time (minutes):  1.1452470699946085
